# dependencies
* install anaconda is recommended

```
cassandra-driver          3.11.0                   py35_1    conda-forge
pandas                    0.19.1              np111py35_0
scikit-learn              0.18.1              np111py35_0
scipy                     0.18.1              np111py35_0
matplotlib                2.0.0               np111py35_0
```

In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import matplotlib
import time
import numpy as np
%matplotlib inline

import pandas as pd
pd.options.display.max_colwidth = 300
pd.options.display.max_columns = 100

# import training as training

In [2]:
#Model File Directory
WORK_DIR = '/drhdev/workspace/ml/'

#Label Keys
LABEL = "success"

MODEL_TYPE = "ML-BR"

start_date = '2018-01-01'
end_date = '2018-03-30'
site_ids = ['avast']
# site_ids = ['avast', 'kasperus', 'adbehap', 'adbehbr', 'adbehkr', 'mcafeeus', 'mfeap', 'mfeeu']
desc = str('%s_%s_%s' % (start_date, end_date, "_".join(site_ids)))

In [3]:
# from cassandra
from cassandra.cluster import Cluster
cassandra_endpoint = '10.224.12.32'
cluster = Cluster([cassandra_endpoint])
# session = cluster.connect('subs')


In [4]:
import datetime
from datetime import timedelta
import time

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import preprocessing
import calendar, datetime
import time
from sklearn.preprocessing import LabelEncoder

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

#Determines the week (number) of the month
def week_of_month(datestr):
    date = to_date(datestr)
    #Calendar object. 6 = Start on Sunday, 0 = Start on Monday
    cal_object = calendar.Calendar(6)
    month_calendar_dates = cal_object.itermonthdates(date.year,date.month)

    day_of_week = 1
    week_number = 1

    for day in month_calendar_dates:
        #add a week and reset day of week
        if day_of_week > 7:
            week_number += 1
            day_of_week = 1

        if date == day:
            break
        else:
            day_of_week += 1

    return str(week_number)

def to_weekday(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date.isoweekday()

def daterange(date1_str, date2_str):
    start_date = to_date(date1_str + ' 00:00:00')
    end_date = to_date(date2_str + ' 00:00:00')
    if start_date > end_date:
        raise ValueError('start date cannot be after the end date.')
    for n in range(int((end_date - start_date).days) + 1):
        yield str(start_date + timedelta(n))

# class PreProcessing(BaseEstimator, TransformerMixin):
#     """Custom Pre-Processing estimator for SRS
#     """

#     def __init__(self):
#         pass

#     def transform(self, df):
#         """Regular transform() that is a help for training, validation & testing datasets
#            (NOTE: The operations performed here are the ones that we did prior to this cell)
#         """
#         # Consolidated feature processing
        
#         df['week_of_month'] = df['transaction_date_in_string'].apply(week_of_month)
#         df['day_of_week'] =  df['transaction_date_in_string'].apply(to_weekday)
#         # one hot for categorical feature ###
# #         df_encoded = pd.get_dummies(df[FEATURES].astype(str), prefix=FEATURES)
# #         df_encoded = pd.DataFrame(df_encoded, columns=self.features_list).fillna(0)

#         df_encoded  = pd.DataFrame(columns=self.features_list)
#         for k, v in self.encoders.items():
#             if df[k].dtype == 'float64':
#                 df[k] = df[k].fillna(-1).astype(int)
            
#             df_encoded[k] = v.transform(df[k].astype(str).str.lower().str.replace(' ',''))
        
#         #Num processing
#         df_num = df[self.FEATURES_NUM].astype(float)
#         df_num = self.scaler.transform(df_num)

#         df_encoded[self.FEATURES_NUM] = df_num

#         return df_encoded.as_matrix()

#     def fit(self, df, y=None, features_dict={}, **fit_params):
        
#         print('features_dict: ', features_dict)
#         self.FEATURES_CAT = features_dict['FEATURES_CAT']
#         self.FEATURES_NUM = features_dict['FEATURES_NUM']
#         self.FEATURES_ENCODED = features_dict['FEATURES_ENCODED']
#         self.FEATURES = self.FEATURES_CAT + self.FEATURES_ENCODED
        
#         df['week_of_month'] = df['transaction_date_in_string'].apply(week_of_month)
#         df['day_of_week'] =  df['transaction_date_in_string'].apply(to_weekday)

        
#         # one hot for categorical feature ###
# #         self.features_list = list(pd.get_dummies(df[FEATURES].astype(str), prefix=FEATURES).columns.values) + FEATURES_NUM

#         self.features_list = self.FEATURES + self.FEATURES_NUM
#         print("self.features_list: ", self.features_list)
#         feature_encoders = {}
#         for f in self.FEATURES:
#             if df[f].dtype == 'float64':
#                 df[f] = df[f].fillna(-1).astype(int)
                
#             encoder = SafeLabelEncoder().fit(df[f].astype(str).str.lower().str.replace(' ',''))
#             feature_encoders[f] = encoder
            
#         self.encoders = feature_encoders  

#         #Fit a scaler
#         df_num = df[self.FEATURES_NUM].astype(float)
#         self.scaler = preprocessing.StandardScaler().fit(df_num)
#         return self


In [43]:
import itertools

subs_session = cluster.connect('subs')
df_cs_original = None
def get_site_date_cartesian_products(site_ids, dates):
    return itertools.product(*[site_ids, dates])

def query_all_sites(dates):
    """ Queries cpg raw data based on the given start, end date and all site_ids """
    query = "SELECT * FROM cpg_transaction where received_date = '%s'"
    _result = pd.DataFrame()
    for date in dates:
        query_by_date = str(query % date.replace('-',''))
        print(query_by_date)
        _result = pd.concat([_result, pd.DataFrame(list(subs_session.execute(query_by_date)))] )
        
    return _result

def query_by_site(dates, site_ids):
    """ Queries cpg raw data based on the given start, end date and list of site_ids """
    query = "SELECT * FROM cpg_transaction where received_date = '%s' and site_id = '%s'"    
    _result = pd.DataFrame()
    for site_id, date in get_site_date_cartesian_products(site_ids, dates):
        query_by_date = str(query % (date.replace('-',''), site_id))
        print(query_by_date)
        _result = pd.concat([_result, pd.DataFrame(list(subs_session.execute(query_by_date)))] )
    
    return _result

def query_raw_data(start, end, site_ids):
    """ Queries cpg raw data based on the given start, end date and list of site_ids """
    
#     _result = pd.DataFrame()
    dates = daterange(start, end)
    
    if not site_ids:
        return query_all_sites(dates)
        
    else:
        return query_by_site(dates, site_ids)

""" Load the raw data into df_cs_original """
df_cs_original = query_raw_data(start_date, end_date, site_ids)
df_cs_original.shape

SELECT * FROM cpg_transaction where received_date = '20180101' and site_id = 'avast'


InvalidRequest: Error from server: code=2200 [Invalid query] message="unconfigured table cpg_transaction"

In [6]:
csv_name = str('df_cs_original_%s_%s_%s.csv' % (start_date, end_date, "_".join(site_ids)))
csv_file_name = WORK_DIR + csv_name
df_cs_original.to_csv(csv_file_name)

csv_file_name

(3398837, 47)

In [6]:
# #Combine two df from csv
# file_1 = 'df_cs_original_2018-01-01_2018-03-31_avast_kasperus_adbehap_adbehbr_adbehkr_mcafeeus_mfeap_mfeeu.csv'
# df_cs_original_1 = pd.read_csv(WORK_DIR + file_1)

# file_2 = 'df_cs_original_2018-04-01_2018-06-30_avast_kasperus_adbehap_adbehbr_adbehkr_mcafeeus_mfeap_mfeeu.csv'


# df_cs_original = pd.concat([df_cs_original_1, pd.read_csv(WORK_DIR + file_2)], axis=0)
# df_cs_original.shape

/drhdev/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (14,15,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/drhdev/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (14,15,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [48]:
df_cs_original['site_id'].value_counts()

avast       3368772
adbehap     1543648
kasperus    1118450
adbehkr      649196
adbehbr      605376
mfeeu        112575
mfeap         17979
mcafeeus        116
Name: site_id, dtype: int64

In [16]:
df_cs_original.head()

,received_date,site_id,subscription_id,event_uuid,added_expiry_years,bank_code,bank_name,billing_country,bin,card_brand,card_category,card_class,card_usage,cc_expiration_date,cid,date_increment,day_of_month,day_of_week,exp_setting_id,funding_source,issuer_country,merchant_descriptor,merchant_number,mid_entity_code,new_status,order_entity_code,payment_amount,payment_amount_usd,payment_currency,payment_method_id,payment_service_id,renew_att_num,request_amount,request_amount_usd,request_currency,requisition_id,response_code,response_message,retry_rule_id,status,subsegment_id,time_of_day_rule_id,transaction_date,transaction_date_in_string,transaction_hour,transaction_id,transaction_type,success
0,20180101,avast,10025882901,4a2d05e6-36b8-491c-8d10-aa79ca6dc399,None,99EBEB6D41E6A44A,"CAPITAL ONE BANK (USA), NATIONAL ASSOCIATION",US,517805,MasterCard,Platinum MasterCard Card,Consumer,True credit (No PIN/Signature capability),0920,None,None,01,Monday,None,Credit,US,DRI*AVAST Software,311009012882,DR_INC-ENTITY,Completed,,85.59,85.59,USD,MasterCard,firstdata,2,85.59,85.59,USD,43984979000,100,00: Approved,None,New,13308769700,,2018-01-01 07:45:45,2018-01-01 01:45:45,01,6333349913,Authorize,1
1,20180101,avast,10036263201,e7510e07-0e46-44b0-a6e3-ff49047df3d5,None,3286758E8A3568CB,Navy Federal Credit Union,US,400022,Visa,Visa Classic,Consumer,Debit Hybrid (PIN and Signature),0120,None,None,01,Monday,None,Debit,US,DRI*AVAST Software,311009012882,DR_INC-ENTITY,Completed,,99.99,99.99,USD,Visa,firstdata,2,99.99,99.99,USD,43981753500,100,00: Approved,None,New,13325277000,,2018-01-01 07:30:06,2018-01-01 01:30:06,01,6333297633,Authorize,1
2,20180101,avast,10039189701,ba44bf00-d656-4515-9488-88024f0fa463,None,NON3DS,Banco de Credito del Peru,PE,455788,Visa,Visa Gold,Consumer,NON USA Consumer Debit - No Pin Access,1220,None,None,01,Monday,None,Debit,PE,DRI*AVAST Software,1156962062,DR_IRELAND-ENTITY,Declined,,179,55.223255,PEN,Visa,netgiro-seb,1,179,55.223255,PEN,43976743900,27001,Transaction refused[51] [000000] [] [],None,New,13331289800,,2018-01-01 07:06:55,2018-01-01 01:06:55,01,6333065673,Authorize,0
3,20180101,avast,10039189701,f2d16301-9c3b-4637-9af0-7a4f0fd09a99,None,NON3DS,Banco de Credito del Peru,PE,455788,Visa,Visa Gold,Consumer,NON USA Consumer Debit - No Pin Access,1220,RETRY_DECLINED.2,None,01,Monday,None,Debit,PE,DRI*AVAST Software,1411163460,DR_IRELAND-ENTITY,Declined,,179,55.223255,PEN,Visa,netgiro-bms,1,179,55.223255,PEN,43976743900,27054,Attempt lower amount.[51] [] [000800] [000800],None,New,13331289800,,2018-01-01 07:06:57,2018-01-01 01:06:57,01,6333066053,Authorize,0
4,20180101,avast,10040113001,3dc16619-c991-4bce-8b54-389ef3eba8b8,None,7B8BE8370D67058F,J.S.C. Federal Credit Union,US,460753,Visa,Visa Classic,Consumer,Debit Hybrid (PIN and Signature),0818,None,None,01,Monday,None,Debit,US,DRI*AVAST Software,311009012882,DR_INC-ENTITY,Completed,,86.59,86.59,USD,Visa,firstdata,1,86.59,86.59,USD,43978651700,100,00: Approved,None,New,13332230000,,2018-01-01 07:18:58,2018-01-01 01:18:58,01,6333186483,Authorize,1


In [13]:
# # Get only data that failed at first attempt

# df_nl = df_cs_original
# # Add 'success' as column
# df_nl['success'] = df_nl['new_status'].map({'Completed':1,'Declined':0, 'Failed':0, 'Reversed':1})
# # Remove renewal data that success at first attempt
# df_nl = df_nl[~((df_nl['renew_att_num'] == 1.0) & (df_nl['success'] == 1))]
# df_nl.shape

(4126194, 49)

In [6]:
# Get only data that failed at first attempt

df_nl = df_cs_original
# Add 'success' as column
df_nl['success'] = df_nl['new_status'].map({'Completed':1,'Declined':0, 'Failed':0, 'Reversed':1})
# Remove renewal data that success at first attempt
df_nl = df_nl[~((df_nl['renew_att_num'] == '1') & (df_nl['success'] == 1))]
df_nl.shape

NameError: name 'df_cs_original' is not defined

In [5]:
import datetime

msg_group = { 'declined' : 'decline', 
             'do_not_honor' : 'do not honor', 
             'txn_refused' : 'refuse', 
             'attempt_lower_amount' : 'lower amount',
            'Insufficient Funds' : 'insufficient',
            'not_allowed' : 'not allowed',
            'correct_cc_retry' : 'correct card',
            'invalid_cc' : 'invalid card',
            'lost_stolen' : 'lost or stolen',
            'invalid_account' : 'invalid account',
            'do_not_try' : 'do not try',
            'expired_card' : 'expired',
            'pickup_card' : 'pick',
            'blocked_first_used' : 'blocked',
            'invalid_txn' : 'invalid trans',
            'restricted_card' : 'restricted',
            'not_permitted' : 'not permitted',
            'expired card' : 'expired card',
            'unable to determine format' : 'determine format',
            'system error' : 'error'
            }

def group_response_msg(msg):
    other = 'Base'
    if isinstance(msg, str) == False:
        return other
    
    msg_lower = msg.lower()
    for key, val in msg_group.items():
        if val in msg_lower:
            return key
        
    return other   

def merge_by_sub(group):

    first = group[ group['renew_att_num'] == '1' ]
    
    if first.empty:
        return pd.DataFrame({})
    
    last = group[ group['renew_att_num'] == group['renew_att_num'].max() ]
        
    first_attempt_date = first['transaction_date_in_string'].iloc[0]
    last_attempt_date = last['transaction_date_in_string'].iloc[0]
    
    last['first_attempt_date'] = first_attempt_date
    last['first_day_of_month'] = first['day_of_month'].iloc[0]
    last['day_of_month'] = last['day_of_month'].astype(int)
    last['first_transaction_hour'] = first['transaction_hour'].iloc[0]
    last['first_response_code'] = first['response_code'].iloc[0]
    
    first_response_msg = first['response_message'].iloc[0]
    last['first_response_message'] = first_response_msg  
    last['first_response_group'] =   first['response_message'].apply(group_response_msg).iloc[0]
    
    last['first_decline_type'] = decline_type(first_response_msg)
    last['days_between'] =  days_between(first_attempt_date, last_attempt_date)
        
    return last

def decline_type(response_msg):
    '''Converts to decline_type based on the given response_msg'''
    dec_type = df_decline_type[df_decline_type['DECLINE_TEXT'] == response_msg]['DECLINE_TYPE']
    if dec_type.empty or dec_type.iloc[0] == 'Base' :
        return group_response_msg(response_msg)
    else:
        return dec_type.iloc[0]

#Convert from str to datetime
def to_date(datestr):
    struct = time.strptime(datestr, "%Y-%m-%d %H:%M:%S")
    date = datetime.date(struct.tm_year,struct.tm_mon,struct.tm_mday)
    return date

#Get difference between d2 and d1 in days.
def days_between(d1, d2):
    d1 = to_date(d1)
    d2 = to_date(d2)
    return abs((d2 - d1).days)

In [6]:
df_decline_type = pd.read_csv(WORK_DIR + 'Decline_Type.csv')

In [ ]:
#Group by subscription_id for failed data at first attempt
time_start = time.time()
df_nl = df_nl[~df_nl['renew_att_num'].isna()]
df_subs_merged = pd.DataFrame()
for site_id in site_ids:
    df_subs = df_nl[df_nl['site_id'] == site_id].groupby(['subscription_id', 'subsegment_id'], sort=False)
    df_subs_merged = pd.concat([df_subs_merged, df_subs.apply(merge_by_sub)], axis=0)
    
print("# merge time:", time.time() - time_start)
df_subs_merged.shape

In [13]:
#Only take subscription that has at least two transactions that occured on different day
retry_success = df_subs_merged[df_subs_merged['days_between'] > 0]

# Drop duplicate columns
retry_success = retry_success.drop(columns=['subscription_id', 'subsegment_id'])

#Write merged subs to csv
csv_name = str('subs_retry_%s_%s_%s.csv' % (start_date, end_date, "_".join(site_ids)))
retry_success.to_csv(WORK_DIR + csv_name)
print("Saved into " + WORK_DIR + csv_name)

Saved into /drhdev/workspace/ml/subs_retry_2018-04-01_2018-06-30_avast_kasperus_adbehap_adbehbr_adbehkr_mcafeeus_mfeap_mfeeu.csv


In [17]:
#Combine two df from csv
df_cs_original = None
file_1 = 'subs_retry_2018-01-01_2018-03-31_avast_kasperus_adbehap_adbehbr_adbehkr_mcafeeus_mfeap_mfeeu.csv'
retry_success_1 = pd.read_csv(WORK_DIR + file_1)

file_2 = 'subs_retry_2018-04-01_2018-06-30_avast_kasperus_adbehap_adbehbr_adbehkr_mcafeeus_mfeap_mfeeu.csv'


retry_success = pd.concat([retry_success_1, pd.read_csv(WORK_DIR + file_2)], axis=0)
retry_success.shape

/drhdev/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/drhdev/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (12,23,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(997957, 57)

In [7]:
retry_success =  pd.read_csv('/drhdev/workspace/ml/pdf_all_site_2018-01-01_2018-06-30.csv')
retry_success.shape

/drhdev/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (37,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(1836209, 57)

In [8]:
retry_success['day_of_week'].value_counts(dropna=False)


monday       276639
wednesday    267770
tuesday      265219
saturday     261475
friday       260917
sunday       260161
thursday     244028
Name: day_of_week, dtype: int64

In [27]:
# retry_success[retry_success['subscription_id'] == 10001280501]
retry_success['success'].value_counts().sort_index()
# retry_success[retry_success['subscription_id'] == 10001280501]
# retry_success.size

0    1534637
1     301572
Name: success, dtype: int64

In [9]:
### resampling, balancing classes ###
def balancing_class(df_ml):
    print(df_ml[LABEL].value_counts(normalize=True))
    df_cls_0 = df_ml[df_ml[LABEL] == 0]
    df_cls_1 = df_ml[df_ml[LABEL] == 1]

    #over sampling
#     df_cls_0_over = df_cls_0.sample(len(df_cls_1), replace=True)
#     df_ml_bl = pd.concat([df_cls_0_over, df_cls_1], axis=0)
    df_cls_1_over = df_cls_1.sample(len(df_cls_0), replace=True)
    df_ml_bl = pd.concat([df_cls_1_over, df_cls_0], axis=0)

#     #under sampling
#     df_cls_1_under = df_cls_1.sample(len(df_cls_0), replace=True)
#     df_ml_bl = pd.concat([df_cls_1_under, df_cls_0], axis=0)
    
    print(df_ml_bl[LABEL].value_counts(normalize=True))
    print(df_ml_bl.shape)
    return df_ml_bl

In [11]:
retry_success_balanced_all = balancing_class(retry_success)
# csv_name = str('subs_retry_balanced__%s_%s_%s.csv' % (start_date, end_date, "_".join(site_ids)))
# retry_success_balanced_all.to_csv(WORK_DIR + csv_name)
# print("Saved into " + WORK_DIR + csv_name)
# retry_success_balanced_all.shape

0    0.835764
1    0.164236
Name: success, dtype: float64
1    0.5
0    0.5
Name: success, dtype: float64
(3069276, 57)


In [12]:
original_size = len(retry_success)
balanced_size = len(retry_success_balanced_all)
fail_size = retry_success[LABEL].value_counts(normalize=True)[0.0]
success_size =  retry_success[LABEL].value_counts(normalize=True)[1.0]

In [13]:
#import for training
import numpy as np
from sklearn import cross_validation
from sklearn import metrics
from sklearn.model_selection import cross_val_score

from sklearn import linear_model
from sklearn import tree
from sklearn import cross_validation
from sklearn import ensemble
from sklearn import linear_model
from sklearn import svm
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import GridSearchCV
# from spark_sklearn import GridSearchCV
from sklearn.preprocessing import label_binarize
from sklearn.neighbors import KNeighborsClassifier

from utils import PreProcessing
from encoder import SafeLabelEncoder

/drhdev/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
# from sklearn.preprocessing import LabelEncoder
# from sklearn.utils.validation import check_is_fitted
# from sklearn.utils import column_or_1d
# # from sklearn.preprocessing.label import _check_numpy_unicode_bug

# def _get_unseen():
#     """Basically just a static method
#     instead of a class attribute to avoid
#     someone accidentally changing it."""
#     return 99999

# class SafeLabelEncoder(LabelEncoder):
#     """An extension of LabelEncoder that will
#     not throw an exception for unseen data, but will
#     instead return a default value of 99999
#     Attributes
#     ----------
#     classes_ : the classes that are encoded
#     """

#     def transform(self, y):
#         """Perform encoding if already fit.
#         Parameters
#         ----------
#         y : array_like, shape=(n_samples,)
#             The array to encode
#         Returns
#         -------
#         e : array_like, shape=(n_samples,)
#             The encoded array
#         """
#         check_is_fitted(self, 'classes_')
#         y = column_or_1d(y, warn=True)

#         classes = np.unique(y)
# #         _check_numpy_unicode_bug(classes)

#         # Check not too many:
#         unseen = _get_unseen()
#         if len(classes) >= unseen:
#             raise ValueError('Too many factor levels in feature. Max is %i' % unseen)

#         e = np.array([
#                          np.searchsorted(self.classes_, x) if x in self.classes_ else unseen
#                          for x in y
#                          ])

#         return e

In [14]:
from sklearn import cross_validation

def print_accuracy_report(classifier, X, y, num_validations=5):
    accuracy = cross_validation.cross_val_score(classifier, 
            X, y, scoring='accuracy', cv=num_validations)
    print("CV Accuracy: " + str(round(100*accuracy.mean(), 2)) + "%")

    f1 = cross_validation.cross_val_score(classifier, 
            X, y, scoring='f1_weighted', cv=num_validations)
    print("CV F1: " + str(round(100*f1.mean(), 2)) + "%")

    precision = cross_validation.cross_val_score(classifier, 
            X, y, scoring='precision_weighted', cv=num_validations)
    print("CV Precision: " + str(round(100*precision.mean(), 2)) + "%")

    recall = cross_validation.cross_val_score(classifier, 
            X, y, scoring='recall_weighted', cv=num_validations)
    print("CV Recall: " + str(round(100*recall.mean(), 2)) + "%")

In [25]:
# FEATURES_CAT = ['card_brand', 'funding_source', 'card_category', 'card_class', 
#                  'issuer_country', 'first_response_code', 'day_of_month',
#                'first_decline_type',  'mid_entity_code', 'payment_service_id', 'payment_method_id', 'bin', 'first_day_of_month']

# FEATURES_NUM = ['payment_amount_usd']
# FEATURES_ENCODED = [ 'week_of_month', 'day_of_week']
# FEATURES = FEATURES_CAT + FEATURES_ENCODED


# # from sklearn.model_selection import GridSearchCV
# from spark_sklearn import GridSearchCV

# # from training import PreProcessing
# from sklearn.pipeline import make_pipeline


# FIELDS = FEATURES_CAT + FEATURES_NUM + ['transaction_date_in_string']

# features_dict = {'FEATURES_CAT': FEATURES_CAT, 'FEATURES_NUM':FEATURES_NUM, 'FEATURES_ENCODED':FEATURES_ENCODED}
# features_dict_key = 'preprocessing__features_dict'
# def display_feature_importance(pipe, model_name, df_features):
#     classifier = pipe.named_steps[model_name]
#     feature_importance = classifier.feature_importances_
#     feature_importance = 100.0 * (feature_importance / feature_importance.max())
#     sorted_idx = np.argsort(feature_importance)
#     print("feature_importance column ",df_features.columns[sorted_idx])
#     print("feature_importance val ",feature_importance[sorted_idx])
#     pos = np.arange(sorted_idx.shape[0]) + .5
#     pvals = feature_importance[sorted_idx]
#     pcols = df_features.columns[sorted_idx]
#     plt.figure(figsize=(8,12))
#     plt.barh(pos, pvals, align='center')
#     plt.yticks(pos, pcols)
#     plt.xlabel('Relative Importance')
#     plt.title('Variable Importance')

# def build_and_train(df, clf, param_grid, model_name, model_file = ''):
#     model_prefix = model_name + '__'
#     time_start = time.time()
#     df_X = df[FIELDS]
 
#     x_train, x_test, y_train, y_test = cross_validation.train_test_split(df_X, df[LABEL], \
#                                                         test_size=0.25, random_state=42)

#     pipe = make_pipeline(PreProcessing(), clf())

#     score = 'accuracy' #  ['accuracy','precision_macro', 'recall_macro', 'f1_macro']

#     print("# Tuning hyper-parameters for %s" % score)
        
#     pipe_param_grid = {model_prefix + k: v for k, v in param_grid.items()}
#     print("pipe_param_grid ", pipe_param_grid)
#     clf_gs = GridSearchCV(sc, pipe, pipe_param_grid, cv=3, scoring=score, n_jobs=-1, fit_params={features_dict_key: features_dict})
# #     clf_gs = GridSearchCV(pipe, pipe_param_grid, cv=5, scoring=score, n_jobs=-1, fit_params={features_dict_key: features_dict})
#     clf_gs.fit(x_train, y_train)

#     print("# Best parameters set found on development set:")
#     print(clf_gs.best_params_)
#     print("# Grid scores on development set:")
#     means = clf_gs.cv_results_['mean_test_score']
#     stds = clf_gs.cv_results_['std_test_score']
#     for mean, std, params in zip(means, stds, clf_gs.cv_results_['params']):
#         print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))

#     print("x_train", x_train.shape)
#     print("x_test", x_test.shape)

#     best_parameters = clf_gs.best_params_
#     best_parameters = {k.replace(model_prefix,''): v for k, v in best_parameters.items()}
#     print("best_parameters ", best_parameters)
#     pipe = make_pipeline(PreProcessing(), clf(**best_parameters))
    
#     if model_name == 'xgbclassifier':
#         print("training xgb ....... ")
#         if model_file == '':
#             pipe.fit(x_train, y_train, preprocessing__features_dict=features_dict, xgbclassifier__eval_metric='error')
#         else:
#             print("Using model_file to train: ", model_file)
#             pipe.fit(x_train, y_train, preprocessing__features_dict=features_dict, xgbclassifier__xgb_model=model_file, xgbclassifier__eval_metric='error')
#     else:        
#         pipe.fit(x_train, y_train, preprocessing__features_dict=features_dict)
        
#     preprocess = pipe.named_steps['preprocessing']

        
#     y_pred_train = pipe.predict(x_train).round()
#     y_pred_test = pipe.predict(x_test).round()
    
#     print("# training time:", time.time() - time_start)

#     clf_d = DummyClassifier(strategy='most_frequent')
#     dummy_pipe = make_pipeline(PreProcessing(), clf_d)
#     dummy_pipe.fit(x_train, y_train, preprocessing__features_dict=features_dict)
#     y_pred_test_dummy = dummy_pipe.predict(x_test).round()
#     print("accuracy_dummy:", metrics.accuracy_score(y_test, y_pred_test_dummy))
#     print("training accuracy:", metrics.accuracy_score(y_train, y_pred_train))
#     print("test accuracy:", metrics.accuracy_score(y_test, y_pred_test))
     
# #     print_accuracy_report(pipe, x_train, y_train, num_validations=3)
#     conf_mx = metrics.confusion_matrix(y_test, y_pred_test)
#     print("# confusion_matrix -  test:\n", conf_mx)
    

#     return pipe


In [22]:
# FEATURES_CAT = ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage',
#                  'issuer_country', 'first_response_code', 'day_of_month', 'site_id', 'billing_country',
#                'first_decline_type',  'merchant_number', 'payment_service_id', 'payment_method_id', 'bin', 'first_day_of_month']

FEATURES_CAT = ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage',
                 'issuer_country', 'first_response_code', 'day_of_month', 'site_id', 'billing_country',
               'first_decline_type',  'merchant_number', 'payment_service_id', 'payment_method_id', 'bin', 'renew_att_num', 'first_day_of_month']

FEATURES_NUM = ['payment_amount_usd']
FEATURES_ENCODED = [ 'week_of_month', 'day_of_week']
FEATURES = FEATURES_CAT + FEATURES_ENCODED


from sklearn.model_selection import GridSearchCV
# from spark_sklearn import GridSearchCV

# from training import PreProcessing
from sklearn.pipeline import make_pipeline


FIELDS = FEATURES_CAT + FEATURES_NUM + ['transaction_date_in_string']

features_dict = {'FEATURES_CAT': FEATURES_CAT, 'FEATURES_NUM':FEATURES_NUM, 'FEATURES_ENCODED':FEATURES_ENCODED}
features_dict_key = 'preprocessing__features_dict'
def display_feature_importance(pipe, model_name, df_features):
    classifier = pipe.named_steps[model_name]
    feature_importance = classifier.feature_importances_
    feature_importance = 100.0 * (feature_importance / feature_importance.max())
    sorted_idx = np.argsort(feature_importance)
    print("feature_importance column ",df_features.columns[sorted_idx])
    print("feature_importance val ",feature_importance[sorted_idx])
    pos = np.arange(sorted_idx.shape[0]) + .5
    pvals = feature_importance[sorted_idx]
    pcols = df_features.columns[sorted_idx]
    plt.figure(figsize=(8,12))
    plt.barh(pos, pvals, align='center')
    plt.yticks(pos, pcols)
    plt.xlabel('Relative Importance')
    plt.title('Variable Importance')

def build_and_train(df, clf, param_grid, model_name, model_file = '', best_param=None):
    model_prefix = model_name + '__'
    time_start = time.time()
    df_X = df[FIELDS]
    result_dict = {}
    x_train, x_test, y_train, y_test = cross_validation.train_test_split(df_X, df[LABEL], \
                                                        test_size=0.25, random_state=42)

    
    if best_param is None:
    
        pipe = make_pipeline(PreProcessing(), clf())
        score = 'accuracy' #  ['accuracy','precision_macro', 'recall_macro', 'f1_macro']
        print("# Tuning hyper-parameters for %s" % score)
        
        pipe_param_grid = {model_prefix + k: v for k, v in param_grid.items()}
        print("pipe_param_grid ", pipe_param_grid)
#         clf_gs = GridSearchCV(sc, pipe, pipe_param_grid, cv=3, scoring=score, n_jobs=-1, fit_params={features_dict_key: features_dict})
        clf_gs = GridSearchCV(pipe, pipe_param_grid, cv=3, scoring=score, n_jobs=-1, fit_params={features_dict_key: features_dict})
        clf_gs.fit(x_train, y_train)

        print("# Best parameters set found on development set:")
        print(clf_gs.best_params_)
        result_dict['hyper_params'] = clf_gs.best_params_
        print("# Grid scores on development set:")
        means = clf_gs.cv_results_['mean_test_score']
        stds = clf_gs.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf_gs.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"% (mean, std * 2, params))

        best_parameters = clf_gs.best_params_
        best_parameters = {k.replace(model_prefix,''): v for k, v in best_parameters.items()}
        
    else:
        best_parameters= best_param
   
    print("best_parameters ", best_parameters)        
    print("x_train", x_train.shape)
    print("x_test", x_test.shape)


    pipe = make_pipeline(PreProcessing(), clf(**best_parameters))
    
    if model_name == 'xgbclassifier':
        print("training xgb ....... ")
        if model_file == '':
            pipe.fit(x_train, y_train, preprocessing__features_dict=features_dict, xgbclassifier__eval_metric='error')
        else:
            print("Using model_file to train: ", model_file)
            pipe.fit(x_train, y_train, preprocessing__features_dict=features_dict, xgbclassifier__xgb_model=model_file, xgbclassifier__eval_metric='error')
    else:        
        pipe.fit(x_train, y_train, preprocessing__features_dict=features_dict)
        
    preprocess = pipe.named_steps['preprocessing']

        
    y_pred_train = pipe.predict(x_train).round()
    y_pred_test = pipe.predict(x_test).round()

    training_time = time.time() - time_start
    print("# training time:", training_time)
    result_dict['training_time'] = training_time
    clf_d = DummyClassifier(strategy='most_frequent')
    dummy_pipe = make_pipeline(PreProcessing(), clf_d)
    dummy_pipe.fit(x_train, y_train, preprocessing__features_dict=features_dict)
    y_pred_test_dummy = dummy_pipe.predict(x_test).round()
    
    accuracy_dummy = metrics.accuracy_score(y_test, y_pred_test_dummy)
    train_accuracy = metrics.accuracy_score(y_train, y_pred_train)
    test_accuracy = metrics.accuracy_score(y_test, y_pred_test)
    
    
    
    
    result_dict['accuracy_dummy'] = accuracy_dummy
    result_dict['train_accuracy'] = train_accuracy
    result_dict['test_accuracy'] = test_accuracy
#     print_accuracy_report(pipe, x_train, y_train, num_validations=3)
#     print(metrics.classification_report(y_test, y_pred_test))
    train_class_report = metrics.classification_report(y_train, y_pred_train)
    test_class_report = metrics.classification_report(y_test, y_pred_test)
    result_dict['train_class_report'] = train_class_report
    result_dict['test_class_report'] = test_class_report
    conf_mx = metrics.confusion_matrix(y_test, y_pred_test)
    
    print("accuracy_dummy:", accuracy_dummy)
    print("training accuracy:", train_accuracy)
    print(train_class_report)
    print("test accuracy:", test_accuracy)
    print(test_class_report)
    print("# confusion_matrix -  test:\n", conf_mx)
    result_dict['conf_mx'] = conf_mx

    return pipe, result_dict


In [16]:
# save model
from sklearn.externals import joblib
import os
from os import path

MODEL_DIR = WORK_DIR + "models"


def write_model(model, model_name, idx=None): 
    build_id = "" if idx is None else "_" + str(idx)
    file_name = path.join(MODEL_DIR, '%s%s.pkl' % (model_name, build_id))
    if not os.path.exists(os.path.dirname(file_name)):
        os.makedirs(os.path.dirname(file_name))
    file = joblib.dump(model, file_name)
    return (file, file_name)

In [17]:
from cassandra.cluster import Cluster
cassandra_endpoint = '10.224.12.32'
cluster = Cluster([cassandra_endpoint])
mlp_session = cluster.connect('mlp')


def insert_model_info(model_id, version, file_name, desc, model_type=MODEL_TYPE, algorithm='XGBClassifier', hyper_parameter=None, eval_metrics=None):
#     file_name = model_id + '.' + str(version) + ".pkl"
    
    mlp_session.execute(
    """
    INSERT INTO ml_model_storage (model_type, model_id, version, features_cat, features_encoded, features_num, repo_path, description, creation_date, modification_date, algorithm, hyper_parameter, eval_metrics)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """,
    (model_type, model_id, version, str(FEATURES_CAT), str(FEATURES_ENCODED), str(FEATURES_NUM), file_name, desc, datetime.datetime.utcnow(), datetime.datetime.utcnow(), algorithm, hyper_parameter, eval_metrics)
        
    )
    print("Model %s version %d is inserted into model repo" % (model_id, version))      


In [18]:
def get_latest_version(model_id, model_type=MODEL_TYPE):
    """Get latest version of the given model_id"""
#     session = cluster.connect('mlp')
    latest_version_query = "select version from ml_model_storage  where model_type = '%s' and model_id = '%s' limit 1" % (model_type, model_id)
    query_result = mlp_session.execute(latest_version_query).one()
    if query_result is None:
        latest_version = 1
    else:
        latest_version = query_result.version
    
    return latest_version


In [19]:
REPOSITORY_URL = 'http://nexus.digitalriverws.net/nexus'
REPO_USER = 'deployment'
REPO_PWD = 'deployment123'
REPO_ID = 'foundationreleases'
REPO_GROUP = 'com.digitalriver.prediction-service'

import repositorytools

def upload_artifact(file_path):
    """Upload artifact to Nexus Repo"""
    artifact = repositorytools.LocalArtifact(local_path=file_path, group=REPO_GROUP)

    client = repositorytools.repository_client_factory(repository_url=REPOSITORY_URL, user=REPO_USER, password=REPO_PWD)
    remote_artifacts = client.upload_artifacts(local_artifacts=[artifact], repo_id=REPO_ID, use_direct_put=True)
    print(remote_artifacts)
    return str(remote_artifacts[0]) if remote_artifacts else ''

In [20]:
""" Train the model"""

#XGBoost Classifier
from xgboost import XGBClassifier
# import xgboost as xgb
classifier = XGBClassifier

tuned_parameters = {
              'objective':['binary:logistic'],
              'learning_rate': [0.2], #so called `eta` value
              'max_depth': [10],
              'min_child_weight': [11],
              'silent': [0],
              'subsample': [0.5],
              'colsample_bytree': [0.7],
#               'n_estimators': [500, 1000], #number of trees, change it to 1000 for better results
              'n_estimators': [1000], #number of trees, change it to 1000 for better results  
              'missing':[-999],
              'seed': [1337]}

best_parameters = {
              'objective': 'binary:logistic',
              'learning_rate': 0.2, #so called `eta` value
              'max_depth': 10,
              'min_child_weight': 11,
              'silent': 0,
              'subsample': 0.5,
              'colsample_bytree': 0.7,
#               'n_estimators': [500, 1000], #number of trees, change it to 1000 for better results
              'n_estimators': 1000, #number of trees, change it to 1000 for better results  
              'missing':-999,
              'seed': 1337}

model_file = ''
# for idx, df_2018_f in enumerate(df_2018_filtered):
#     print("Iteration: ", idx)
model_id = 'ML-BR-1'
version = get_latest_version(model_id) + 1
model_name = model_id + '.' + str(version)
xgb_clf, result_d = build_and_train(retry_success_balanced_all, classifier, tuned_parameters, 'xgbclassifier', model_file, best_param=best_parameters)
model_file, model_file_name = write_model(xgb_clf, model_name)
print("model_file is generated: ", model_file)
print("result_dict: ", result_d)


best_parameters  {'objective': 'binary:logistic', 'learning_rate': 0.2, 'max_depth': 10, 'min_child_weight': 11, 'silent': 0, 'subsample': 0.5, 'colsample_bytree': 0.7, 'n_estimators': 1000, 'missing': -999, 'seed': 1337}
x_train (2301957, 18)
x_test (767319, 18)
training xgb ....... 
features_dict:  {'FEATURES_CAT': ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'issuer_country', 'first_response_code', 'day_of_month', 'site_id', 'billing_country', 'first_decline_type', 'merchant_number', 'payment_service_id', 'payment_method_id', 'bin', 'renew_att_num'], 'FEATURES_NUM': ['payment_amount_usd'], 'FEATURES_ENCODED': ['week_of_month', 'day_of_week']}
self.features_list:  ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'issuer_country', 'first_response_code', 'day_of_month', 'site_id', 'billing_country', 'first_decline_type', 'merchant_number', 'payment_service_id', 'payment_method_id', 'bin', 'renew_att_num', 'week_of_month', 'd

In [23]:
""" Train the model"""

#XGBoost Classifier
from xgboost import XGBClassifier
# import xgboost as xgb
classifier = XGBClassifier

tuned_parameters = {
              'objective':['binary:logistic'],
              'learning_rate': [0.2], #so called `eta` value
              'max_depth': [10],
              'min_child_weight': [11],
              'silent': [0],
              'subsample': [0.5],
              'colsample_bytree': [0.7],
              'n_estimators': [1000], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [1337]}

best_parameters = {
              'objective': 'binary:logistic',
              'learning_rate': 0.2, #so called `eta` value
              'max_depth': 10,
              'min_child_weight': 11,
              'silent': 0,
              'subsample': 0.5,
              'colsample_bytree': 0.7,
              'n_estimators': 1000, #number of trees, change it to 1000 for better results  
              'missing':-999,
              'seed': 1337}

model_file = ''

model_id = 'ML-BR-1'
version = get_latest_version(model_id) + 1
model_name = model_id + '.' + str(version)
xgb_clf, result_d = build_and_train(retry_success_balanced_all, classifier, tuned_parameters, 'xgbclassifier', model_file, best_param=best_parameters)
model_file, model_file_name = write_model(xgb_clf, model_name)
print("model_file is generated: ", model_file)
print("result_dict: ", result_d)


best_parameters  {'objective': 'binary:logistic', 'learning_rate': 0.2, 'max_depth': 10, 'min_child_weight': 11, 'silent': 0, 'subsample': 0.5, 'colsample_bytree': 0.7, 'n_estimators': 1000, 'missing': -999, 'seed': 1337}
x_train (2301957, 19)
x_test (767319, 19)
training xgb ....... 
features_dict:  {'FEATURES_CAT': ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'issuer_country', 'first_response_code', 'day_of_month', 'site_id', 'billing_country', 'first_decline_type', 'merchant_number', 'payment_service_id', 'payment_method_id', 'bin', 'renew_att_num', 'first_day_of_month'], 'FEATURES_NUM': ['payment_amount_usd'], 'FEATURES_ENCODED': ['week_of_month', 'day_of_week']}
self.features_list:  ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'issuer_country', 'first_response_code', 'day_of_month', 'site_id', 'billing_country', 'first_decline_type', 'merchant_number', 'payment_service_id', 'payment_method_id', 'bin', 'renew_att_num

In [30]:
""" Train the model"""

#XGBoost Classifier
from xgboost import XGBClassifier
# import xgboost as xgb
classifier = XGBClassifier

tuned_parameters = {
              'objective':['binary:logistic'],
              'learning_rate': [0.2], #so called `eta` value
              'max_depth': [10],
              'min_child_weight': [11],
              'silent': [0],
              'subsample': [0.5],
              'colsample_bytree': [0.7],
              'n_estimators': [1000], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [1337]}

best_parameters = {
              'objective': 'binary:logistic',
              'learning_rate': 0.2, #so called `eta` value
              'max_depth': 10,
              'min_child_weight': 11,
              'silent': 0,
              'subsample': 0.5,
              'colsample_bytree': 0.7,
              'n_estimators': 1000, #number of trees, change it to 1000 for better results  
              'missing':-999,
              'seed': 1337}

model_file = ''

model_id = 'ML-BR-1'
version = get_latest_version(model_id) + 1
model_name = model_id + '.' + str(version)
xgb_clf, result_d = build_and_train(retry_success_balanced_all, classifier, tuned_parameters, 'xgbclassifier', model_file, best_param=best_parameters)
model_file, model_file_name = write_model(xgb_clf, model_name)
print("model_file is generated: ", model_file)
print("result_dict: ", result_d)


best_parameters  {'objective': 'binary:logistic', 'learning_rate': 0.2, 'max_depth': 10, 'min_child_weight': 11, 'silent': 0, 'subsample': 0.5, 'colsample_bytree': 0.7, 'n_estimators': 1000, 'missing': -999, 'seed': 1337}
x_train (2301955, 18)
x_test (767319, 18)
training xgb ....... 
features_dict:  {'FEATURES_CAT': ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'issuer_country', 'first_response_code', 'day_of_month', 'site_id', 'billing_country', 'first_decline_type', 'merchant_number', 'payment_service_id', 'payment_method_id', 'bin', 'renew_att_num'], 'FEATURES_NUM': ['payment_amount_usd'], 'FEATURES_ENCODED': ['week_of_month', 'day_of_week']}
self.features_list:  ['card_brand', 'funding_source', 'card_category', 'card_class', 'card_usage', 'issuer_country', 'first_response_code', 'day_of_month', 'site_id', 'billing_country', 'first_decline_type', 'merchant_number', 'payment_service_id', 'payment_method_id', 'bin', 'renew_att_num', 'week_of_month', 'd

In [24]:
"""Upload model to Nexus repo and insert the model info into Cassandra table"""

repo_path = upload_artifact(model_file_name)
size_desc = str(", original size: %s (fail: %s, success: %s), balanced_size: %s" % (original_size, fail_size, success_size, balanced_size))
desc = desc + size_desc
hyper_params = result_d.pop('hyper_params', None)
insert_model_info(model_id, version, repo_path, desc=desc, eval_metrics=str(result_d), 
                  hyper_parameter=str(hyper_params))

The following files were uploaded to repository foundationreleases


http://nexus.digitalriverws.net/nexus/content/repositories/foundationreleases/com/digitalriver/prediction-service/ML-BR/1.34/ML-BR-1.34.pkl
[com.digitalriver.prediction-service:ML-BR:1.34::pkl]
Model ML-BR-1 version 34 is inserted into model repo
